In [75]:

import rasterio
import matplotlib
import matplotlib.pyplot as pyplot
from matplotlib.colors import ListedColormap
from functional import seq
import numpy


Show raster

In [ ]:
with rasterio.open("./data/CONUS_Landcover_B1/CONUS_B1_y2030.tif") as src:
  band1 = src.read(1) # as a numpy array
  
  # get the colormap for the band
  embedded_colormap: dict = src.colormap(1)  
  colors = seq(list(embedded_colormap.values())).map(lambda color: tuple(item / 255 for item in color)).list()
  colormap = ListedColormap(colors[0:18]) # do not include unused values or pyplot will not be able to plot the colors correctly
  
  # show nunmpy array as a raster image
  pyplot.imshow(band1, cmap=colormap)
  pyplot.show()

In [ ]:
with rasterio.open("./data/CONUS_Landcover_B1/CONUS_B1_y2030.tif") as src:
  out_meta = src.meta
  
  band1 = src.read(1) # as a numpy array
  
  # create new array where all values in the band of 2 becomes 1 and everything else becomes 0
  # in order to create an array that shows where developed land is located
  recalc = numpy.where(band1 == 2, 1, 0)
  
with rasterio.open("/mnt/d/output.tif", 'w', **out_meta) as dataset:
  dataset.write(recalc, 1)
  

In [ ]:
# open the raster and lock it in the filesystem while working on it
raster = rasterio.open("./data/CONUS_Landcover_B1/CONUS_B1_y2030.tif")

e = [2]



# create new array where all values in the band of 2 becomes 1 and everything else becomes 0
# in order to create an array that shows where developed land is located
band1 = raster.read(1)
recalc = numpy.where(numpy.isin(band1, e), 1, 0)

pyplot.imshow(recalc)
pyplot.show()

# export the extracted band pixel values
# with the extracted band as black pixels and the rest as transparent white pixels
out_profile = raster.profile.copy()
out_profile.update(nodata=0)
with rasterio.open("/mnt/d/output.tif", "w", **out_profile) as dest:
   dest.write(recalc, 1)
   dest.write_colormap(1, { 1: (255, 0, 0, 255) })

# remove the lock on the raster
raster.close()